In [1]:
import numpy as np
import pandas as pd

from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score

import string
import warnings
import missingno
warnings.filterwarnings('ignore')


import pandas as pd
import cufflinks as cf
import chart_studio.plotly as py
import plotly.graph_objects as go
import numpy as np
import pandas as pd
import numpy as np
from numpy.random import gamma
from numpy.random import multivariate_normal
from scipy.stats import multivariate_t
from scipy.stats import f
import matplotlib.pyplot as plt



cf.go_offline(connected = True)


import pandas as pd
import cufflinks as cf
import chart_studio.plotly as py
import plotly.graph_objects as go
import numpy as np
import pandas as pd
import numpy as np
from numpy.random import gamma
from numpy.random import multivariate_normal
from scipy.stats import multivariate_t
from scipy.stats import f
import matplotlib.pyplot as plt
import missingno


cf.go_offline(connected = True)


import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

from sklearn.linear_model import Ridge, Lasso
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_squared_log_error
from sklearn.metrics import make_scorer
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

import numpy as np
import pandas as pd
from sklearn.metrics import make_scorer
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor 
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV
import string
import warnings
import missingno
from scipy.stats import f
import scipy

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

D:\anaconda\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
df_용역 = pd.read_csv('용역_Proj_02_06.csv').iloc[:,1:]
df_공사 = pd.read_csv('공사_Proj_02_06.csv').iloc[:,1:]
df_물품 = pd.read_csv('물품_Proj_02_06.csv').iloc[:,1:]

df_용역['공고게시일자'] = pd.to_datetime(df_용역['공고게시일자'])
df_공사['공고게시일자'] = pd.to_datetime(df_공사['공고게시일자'])
df_물품['공고게시일자'] = pd.to_datetime(df_물품['공고게시일자'])

df_물품_추정가격 = df_물품.groupby(pd.Grouper(key = '공고게시일자', freq = '1W')).sum()['추정가격']
df_공사_추정가격 = df_공사.groupby(pd.Grouper(key = '공고게시일자', freq = '1W')).sum()['추정가격']
df_용역_추정가격 = df_용역.groupby(pd.Grouper(key = '공고게시일자', freq = '1W')).sum()['추정가격']

df = pd.merge(df_용역_추정가격,df_공사_추정가격,left_index= True, right_index=True, how = 'outer')
df_total_추정가격 = pd.merge(df,df_물품_추정가격,left_index= True, right_index=True , how = 'outer')

df_total_추정가격.columns = ['용역','공사','물품']

df_test = df_total_추정가격.reset_index()[['용역']].reset_index()

df_물품_ZZ = df_물품[df_물품['수요기관코드'] == 'B500001']
df_용역_ZZ = df_용역[df_용역['수요기관코드'] == 'B500001']
df_공사_ZZ = df_공사[df_공사['수요기관코드'] == 'B500001']

df_물품_ZZ_grouper = df_물품_ZZ.groupby(pd.Grouper(key = '공고게시일자', freq = '1W')).sum()[['추정가격']]
df_용역_ZZ_grouper = df_용역_ZZ.groupby(pd.Grouper(key = '공고게시일자', freq = '1W')).sum()[['추정가격']]
df_공사_ZZ_grouper = df_공사_ZZ.groupby(pd.Grouper(key = '공고게시일자', freq = '1W')).sum()[['추정가격']]

df_물품_ZZ_grouper.reset_index()[['추정가격']]
df_용역_ZZ_grouper.reset_index()[['추정가격']]
df_공사_ZZ_grouper.reset_index()[['추정가격']]

df_test = pd.merge(df_물품_ZZ_grouper.reset_index()[['추정가격']],df_용역_ZZ_grouper.reset_index()[['추정가격']],left_index = True, right_index =True, how = 'outer')
df_test_2 = pd.merge(df_test,df_용역_ZZ_grouper.reset_index()[['추정가격']],left_index = True, right_index =True, how = 'outer')

df_test.columns = ['물품','용역']
df_test_2.columns = ['물품','용역','공사']

df_test = df_test.fillna(0).copy()
df_test_2 = df_test_2.fillna(0).copy()

In [3]:
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import robust_scale

transformer  = RobustScaler()
transformer.fit(df_test)
df_test = transformer.transform(df_test)

transformer  = RobustScaler()
transformer.fit(df_test_2)
df_test_2 = transformer.transform(df_test_2)

In [4]:
# 적절한 gamma 값 ; sklearn의 svm에서 기본 default값 계산을 참조함 (2차원)
1/ (2 *np.array(df_test).var())

0.17349953467545662

In [5]:
# 적절한 gamma 값 ; sklearn의 svm에서 기본 default값 계산을 참조함 (3차원)
1/ (2 *np.array(df_test_2).var()) 

0.15141959537014263

In [6]:
import sys
sys.path.append("C:/Users/USER/Documents/Jupyter Notebook/public_procurement/Project_02_15\SVDD-Python-master/")

## 일단 2차원(물품 / 용역) 만 보기로 하였다. 왜냐하면, 시각화 시켜서 이해하기에는 2차원이 적당하기 때문이다.

In [7]:
pd.DataFrame(df_test)

,0,1
0,-0.580383,-0.476234
1,-0.581019,-0.476234
2,-0.672586,-0.476234
3,-0.613643,-0.476234
4,-0.352531,-0.476234
...,...,...
202,5.311560,-0.476234
203,4.543185,-0.476234
204,2.616027,-0.476234
205,-0.325642,-0.476234


In [9]:
"""
An example for SVDD model fitting with negataive samples
"""

import numpy as np
from src.BaseSVDD import BaseSVDD

# create 100 points with 2 dimensions
# n = 100
# dim = 2
# X = np.r_[np.random.randn(n, dim)]

X = np.array(df_test)

# svdd object using rbf kernel
svdd = BaseSVDD(C=1.0, gamma=0.129, kernel='rbf', display='on')

# fit the SVDD model
svdd.fit(X)

# predict the label
y_predict = svdd.predict(X)

# plot the boundary
svdd.plot_boundary(X)

# plot the distance
radius = svdd.radius
distance = svdd.get_distance(X)
svdd.plot_distance(radius, distance)

TypeError: fit() missing 2 required positional arguments: 'bs_size' and 'alpha'

## 3차원으로 실행하였을 때는 다음과 같다. 이때는 4차원으로 차원을 높이는데 시각화할 수 없으므로, K 관리도 만 그리기로 하였다.

In [ ]:

"""
An example for SVDD model fitting with negataive samples
"""
import sys
sys.path.append("C:/Users/USER/Documents/Jupyter Notebook/public_procurement/Project_02_15\SVDD-Python-master/")
import numpy as np
from src.BaseSVDD import BaseSVDD

# create 100 points with 2 dimensions
# n = 100
# dim = 2
# X = np.r_[np.random.randn(n, dim)]

X = np.array(df_test_2)

# svdd object using rbf kernel
svdd = BaseSVDD(C=1.0, gamma=0.15, kernel='rbf', display='on')

# fit the SVDD model
svdd.fit(X)

# predict the label
y_predict = svdd.predict(X)

# plot the boundary
# svdd.plot_boundary(X)

# # plot the distance
radius = svdd.radius
distance = svdd.get_distance(X)
svdd.plot_distance(radius, distance)

## PSO 는 최적점을 찾아가는 과정으로, 기존 Gradient descent에서 multi-point를 보고 진행하는 것이다.
## 이때 주어지는 데이터는 임의로 설정한 데이터이다. X가 input이고, y가 one-class 이다. 여기서 SVDD가 PSO를 통해서  얼마나 잘 이상치를 감지해 나가는지 확인하는 과정이다.

In [ ]:

from src.BaseSVDD import BaseSVDD, BananaDataset
from sko.PSO import PSO
import matplotlib.pyplot as plt


# Banana-shaped dataset generation and partitioning
X, y = BananaDataset.generate(number=100, display='off')
X_train, X_test, y_train, y_test = BananaDataset.split(X, y, ratio=0.3)

# objective function
def objective_func(x):
    x1, x2 = x
    svdd = BaseSVDD(C=x1, gamma=x2, kernel='rbf', display='off')
    y = 1-svdd.fit(X_train, y_train).accuracy
    return y

# Do PSO
pso = PSO(func=objective_func, dim=2, pop=10, max_iter=20, 
          lb=[0.01, 0.01], ub=[1, 3], w=0.8, c1=0.5, c2=0.5)
pso.run()

print('best_x is', pso.gbest_x)
print('best_y is', pso.gbest_y)

# plot the result

fig = go.Figure()
fig.add_trace(go.Scatter(x = pd.DataFrame([i for i in range(len(pso.gbest_y_hist))])[0],y = pd.DataFrame(pso.gbest_y_hist)[0], mode = 'lines+markers'))
fig.update_layout(
    autosize = False,
    width = 800,
    height = 800
)


fig.show()

In [ ]:
import sys
sys.path.append("..")
from sklearn.datasets import load_wine
from src.BaseSVDD import BaseSVDD, BananaDataset

# Banana-shaped dataset generation and partitioning
X, y = BananaDataset.generate(number=100, display='on')
X_train, X_test, y_train, y_test = BananaDataset.split(X, y, ratio=0.3)

# 
svdd = BaseSVDD(C=0.10905555, gamma=0.60239708, kernel='rbf', display='on')

# 
svdd.fit(X_train,  y_train)

# 
svdd.plot_boundary(X_train,  y_train)

#
y_test_predict = svdd.predict(X_test, y_test)

#
radius = svdd.radius
distance = svdd.get_distance(X_test)
svdd.plot_distance(radius, distance)

## PCO방식은 Hyper-parameter을 찾는 과정 중에 하나이고, 위는 이 방식을 통해서 찾은 Hyper-parameter을 임의의 데이터(Banana 모양)에 적용시키고, 그에 대해서 Train Data 에 대한 정확도와 Test 데이터에 대한 정확도를 보는 것이다 (KFold 사용 x)

## 이번에는 위의 데이터에 대해서 K-Fold 방식으로 정확도를 계산하여 보자.

In [ ]:
import sys
sys.path.append("..")
from src.BaseSVDD import BaseSVDD, BananaDataset
from sklearn.model_selection import cross_val_score


# Banana-shaped dataset generation and partitioning
X, y = BananaDataset.generate(number=100, display='on')
X_train, X_test, y_train, y_test = BananaDataset.split(X, y, ratio=0.3)

# 
svdd = BaseSVDD(C=0.9, gamma=0.3, kernel='rbf', display='on')


# cross validation (k-fold)
k = 5
scores = cross_val_score(svdd, X_train, y_train, cv=k, scoring='accuracy')

#
print("Cross validation scores:")
for scores_ in scores:
    print(scores_)
 
print("Mean cross validation score: {:4f}".format(scores.mean()))

## PCO와 다르게, Grid_Search로 (정확도를 performance로 봄) Hyper-parameter을 보도록 하자.

In [ ]:
import sys
sys.path.append("..")
from sklearn.datasets import load_wine
from src.BaseSVDD import BaseSVDD, BananaDataset
from sklearn.model_selection import KFold, LeaveOneOut, ShuffleSplit
from sklearn.model_selection import learning_curve, GridSearchCV

# Banana-shaped dataset generation and partitioning
X, y = BananaDataset.generate(number=100, display='off')
X_train, X_test, y_train, y_test = BananaDataset.split(X, y, ratio=0.3)

param_grid = [
    {"kernel": ["rbf"], "gamma": [0.1, 0.2, 0.5], "C": [0.1, 0.5, 1]},
    {"kernel": ["linear"], "C": [0.1, 0.5, 1]},
    {"kernel": ["poly"], "C": [0.1, 0.5, 1], "degree": [2, 3, 4, 5]},
]

svdd = GridSearchCV(BaseSVDD(display='off'), param_grid, cv=5, scoring="accuracy")
svdd.fit(X_train, y_train)
print("best parameters:")
print(svdd.best_params_)
print("\n")

# 
best_model = svdd.best_estimator_
means = svdd.cv_results_["mean_test_score"]
stds = svdd.cv_results_["std_test_score"]

for mean, std, params in zip(means, stds, svdd.cv_results_["params"]):
    print("%0.3f (+/-%0.03f) for %r" % (mean, std * 2, params))
print()

## 이번에는 데이터를 plot 시켜보았을 때 비선형 이라면 어떻게 되는지 보고 싶다. KPCA 라는 방법을 사용하였으며, Test, Train으로 나누어지지 않았으므로, Prediction과 Train의 accuracy가 동일하다

In [ ]:
import sys
sys.path.append("..")
import numpy as np
from src.BaseSVDD import BaseSVDD
from sklearn.decomposition import KernelPCA


# create 100 points with 5 dimensions
X = np.r_[np.random.randn(50, 5) + 1, np.random.randn(50, 5)]
y = np.append(np.ones((50, 1), dtype=np.int64), 
              -np.ones((50, 1), dtype=np.int64),
              axis=0)

# number of the dimensionality
kpca = KernelPCA(n_components=2, kernel="rbf", gamma=0.1, fit_inverse_transform=True)
X_kpca = kpca.fit_transform(X)

# fit the SVDD model
svdd = BaseSVDD(C=0.9, gamma=10, kernel='rbf', display='on')

# fit and predict
svdd.fit(X_kpca,  y)
y_test_predict = svdd.predict(X_kpca, y)

# plot the distance curve
radius = svdd.radius
distance = svdd.get_distance(X_kpca)
svdd.plot_distance(radius, distance)

# plot the boundary
svdd.plot_boundary(X_kpca,  y)

## 임의의 데이터을 기반으로 사용한 모델에 대해서 Confusion matrix 및 ROC 커브를 그려보았다.

In [ ]:
"""
An example for drawing the confusion matrix and ROC curve
"""
import sys
sys.path.append("..")
import numpy as np
import matplotlib.pyplot as plt
from src.BaseSVDD import BaseSVDD
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import train_test_split

# generate data
n = 100
dim = 5
X = np.r_[np.random.randn(n, dim) + 1, np.random.randn(n, dim)]
y = np.append(np.ones((n, 1), dtype=np.int64), 
              -np.ones((n, 1), dtype=np.int64),
              axis=0)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=0)

# SVDD model
svdd = BaseSVDD(C=0.12, gamma=0.1, kernel='rbf', display='on')
svdd.fit(X_train,  y_train)
y_test_predict = svdd.predict(X_test, y_test)

# plot the distance curve
radius = svdd.radius
distance = svdd.get_distance(X_test)
svdd.plot_distance(radius, distance)

# confusion matrix and ROC curve
cm = confusion_matrix(y_test, y_test_predict)
cm_display = ConfusionMatrixDisplay(cm).plot()
y_score = svdd.decision_function(X_test)

fpr, tpr, _ = roc_curve(y_test, y_score)
roc_auc = auc(fpr, tpr)

plt.figure()
plt.plot(fpr, tpr, color="darkorange", lw=3, label="ROC curve (area = %0.2f)" % roc_auc)
plt.plot([0, 1], [0, 1], color="navy", lw=3, linestyle="--")
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("Receiver operating characteristic")
plt.legend(loc="lower right")
plt.grid()
plt.show()
